In [36]:
!sudo apt-get install python2.7

!pip install datasets

!git clone https://github.com/nusnlp/m2scorer.git
!/usr/bin/python2.7 ./m2scorer/m2scorer m2scorer/example/system m2scorer/example/source_gold

from datasets import load_dataset, load_metric

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python2.7 is already the newest version (2.7.18-13ubuntu1.2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
fatal: destination path 'm2scorer' already exists and is not an empty directory.
Precision   : 0.8000
Recall      : 0.8000
F_0.5       : 0.8000


In [7]:
raw_datasets = load_dataset("wi_locness", "wi")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/300 [00:00<?, ? examples/s]

In [24]:
def apply_edits_to_text(example):
    original_text = example["text"]
    corrected_text = example["text"]
    for start, end, correction in reversed(list(zip(
        example["edits"]["start"],
        example["edits"]["end"],
        example["edits"]["text"]))):
        if correction == None:
            continue
        corrected_text = corrected_text[:start] + correction + corrected_text[end:]
    return original_text, corrected_text

In [ ]:
item = raw_datasets['train'][0]
original_sentence = item['text']
edits = item['edits']

print(original_sentence)
print(edits)

corrected_sentence = apply_edits_to_text(item)
print(corrected_sentence)

with open('corrected_outputs.txt', 'w') as file:
    file.write(corrected_sentence[1])

My town is a medium size city with eighty thousand inhabitants. It has a high density population because its small territory. Despite of it is an industrial city, there are many shops and department stores.  I recommend visiting the artificial lake in the certer of the city which is surrounded by a park. Pasteries are very common and most of them offer the special dessert from the city. There are a comercial zone along the widest street of the city where you can find all kind of establishments: banks, bars, chemists, cinemas, pet shops, restaurants, fast food restaurants, groceries, travel agencies, supermarkets and others. Most of the shops have sales and offers at least three months of the year: January, June and August. The quality of the products and services are quite good, because there are a huge competition, however I suggest you taking care about some fakes or cheats.
{'start': [13, 77, 104, 126, 134, 256, 306, 375, 396, 402, 476, 484, 579, 671, 774, 804, 808, 826, 838, 850, 8

In [32]:
import subprocess
import tempfile
import os
from tqdm import tqdm

gold_edits_file = 'gold_edits_test.txt'

dataset = raw_datasets['train'].select(range(1))

for row in tqdm(dataset, desc="Processing rows"):
    original_text, corrected_text = apply_edits_to_text(row)

    with tempfile.NamedTemporaryFile(mode='w+', delete=False) as orig_file, \
         tempfile.NamedTemporaryFile(mode='w+', delete=False) as corr_file:

        orig_file.write(original_text)
        corr_file.write(corrected_text)

        orig_file.flush()
        corr_file.flush()

        # Call the edit_creator.py script with the temporary file paths
        output_file = tempfile.NamedTemporaryFile(mode='w+', delete=False)
        command = ["/usr/bin/python2.7", "./m2scorer/scripts/edit_creator.py", "--output", output_file.name, orig_file.name, corr_file.name]
        subprocess.run(command)

        # Append the output to the gold edits file
        output_file.seek(0)
        with open(gold_edits_file, 'a') as gold_file:
            gold_file.write(output_file.read())

    # Clean up the temporary files
    os.remove(orig_file.name)
    os.remove(corr_file.name)
    os.remove(output_file.name)

Processing rows: 100%|██████████| 1/1 [00:00<00:00,  2.47it/s]


In [35]:
!/usr/bin/python2.7 ./m2scorer/m2scorer corrected_outputs.txt gold_edits_test.txt

Precision   : 1.0000
Recall      : 1.0000
F_0.5       : 1.0000
